In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import logging

In [3]:
# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Define the InceptionBlock and InceptionLSTMMoonquake classes as provided
class InceptionBlock(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(InceptionBlock, self).__init__()
        
        self.branch1 = nn.Conv1d(in_channels, out_channels, kernel_size=1)
        
        self.branch2 = nn.Sequential(
            nn.Conv1d(in_channels, out_channels, kernel_size=1),
            nn.Conv1d(out_channels, out_channels, kernel_size=3, padding=1)
        )
        
        self.branch3 = nn.Sequential(
            nn.Conv1d(in_channels, out_channels, kernel_size=1),
            nn.Conv1d(out_channels, out_channels, kernel_size=5, padding=2)
        )
        
        self.branch4 = nn.Sequential(
            nn.MaxPool1d(kernel_size=3, stride=1, padding=1),
            nn.Conv1d(in_channels, out_channels, kernel_size=1)
        )
    
    def forward(self, x):
        b1 = self.branch1(x)
        b2 = self.branch2(x)
        b3 = self.branch3(x)
        b4 = self.branch4(x)
        return torch.cat([b1, b2, b3, b4], 1)

In [4]:
class InceptionLSTMMoonquake(nn.Module):
    def __init__(self, sequence_length=1000, num_classes=1):
        super(InceptionLSTMMoonquake, self).__init__()
        
        self.inception1 = InceptionBlock(1, 32)
        self.inception2 = InceptionBlock(128, 32)
        
        self.lstm = nn.LSTM(
            input_size=128,
            hidden_size=64,
            num_layers=2,
            batch_first=True,
            dropout=0.2
        )
        
        self.fc = nn.Sequential(
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(32, num_classes),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        x = x.unsqueeze(1)
        
        x = self.inception1(x)
        x = self.inception2(x)
        
        x = x.permute(0, 2, 1)
        x, _ = self.lstm(x)
        x = x[:, -1, :]
        
        x = self.fc(x)
        return x

# Custom Dataset class
class MoonquakeDataset(Dataset):
    def __init__(self, signals, labels):
        self.signals = torch.FloatTensor(signals)
        self.labels = torch.FloatTensor(labels)
    
    def __len__(self):
        return len(self.signals)
    
    def __getitem__(self, idx):
        return self.signals[idx], self.labels[idx]

# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs, device):
    best_val_loss = float('inf')
    best_model = None
    
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for signals, labels in train_loader:
            signals, labels = signals.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(signals)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
        
        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for signals, labels in val_loader:
                signals, labels = signals.to(device), labels.to(device)
                outputs = model(signals)
                val_loss += criterion(outputs, labels).item()
        
        train_loss /= len(train_loader)
        val_loss /= len(val_loader)
        
        logger.info(f'Epoch {epoch+1}/{num_epochs}:')
        logger.info(f'Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}')
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_model = model.state_dict().copy()
    
    return best_model

# Evaluation function
def evaluate_model(model, test_loader, device):
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for signals, labels in test_loader:
            signals, labels = signals.to(device), labels.to(device)
            outputs = model(signals)
            preds = (outputs > 0.5).float()
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)
    
    accuracy = accuracy_score(all_labels, all_preds)
    precision = precision_score(all_labels, all_preds)
    recall = recall_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds)
    
    return accuracy, precision, recall, f1

# Inference function for deployment
def inference(model, signal, device):
    model.eval()
    with torch.no_grad():
        signal = torch.FloatTensor(signal).unsqueeze(0).to(device)
        output = model(signal)
        is_quake = output.item() > 0.5
    return is_quake


In [5]:
# Main function to tie everything together
def main():
    # Hyperparameters
    SEQUENCE_LENGTH = 1000
    BATCH_SIZE = 32
    NUM_EPOCHS = 50
    LEARNING_RATE = 0.001
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Load and preprocess data
    # This is a placeholder - replace with actual data loading
    signals = np.random.randn(1000, SEQUENCE_LENGTH)  # Replace with real data
    labels = np.random.randint(0, 2, (1000, 1))      # Replace with real labels
    
    # Split data
    train_signals, test_signals, train_labels, test_labels = train_test_split(
        signals, labels, test_size=0.2, random_state=42
    )
    train_signals, val_signals, train_labels, val_labels = train_test_split(
        train_signals, train_labels, test_size=0.2, random_state=42
    )
    
    # Create datasets and dataloaders
    train_dataset = MoonquakeDataset(train_signals, train_labels)
    val_dataset = MoonquakeDataset(val_signals, val_labels)
    test_dataset = MoonquakeDataset(test_signals, test_labels)
    
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
    test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)
    
    # Initialize model, criterion, and optimizer
    model = InceptionLSTMMoonquake(sequence_length=SEQUENCE_LENGTH).to(DEVICE)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
    
    # Train model
    best_model_state = train_model(model, train_loader, val_loader, criterion, optimizer, NUM_EPOCHS, DEVICE)
    
    # Load best model and evaluate
    model.load_state_dict(best_model_state)
    accuracy, precision, recall, f1 = evaluate_model(model, test_loader, DEVICE)
    
    logger.info(f'Test Metrics:')
    logger.info(f'Accuracy: {accuracy:.4f}')
    logger.info(f'Precision: {precision:.4f}')
    logger.info(f'Recall: {recall:.4f}')
    logger.info(f'F1 Score: {f1:.4f}')
    
    # Save model for deployment
    torch.save(model.state_dict(), 'moonquake_detector.pth')
    logger.info('Model saved successfully')

# Function for deployment on satellite
def satellite_inference(signal):
    DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = InceptionLSTMMoonquake().to(DEVICE)
    model.load_state_dict(torch.load('moonquake_detector.pth'))
    
    is_quake = inference(model, signal, DEVICE)
    return is_quake

if __name__ == "__main__":
    main()

INFO:__main__:Epoch 1/50:
INFO:__main__:Train Loss: 0.6944, Val Loss: 0.6965
INFO:__main__:Epoch 2/50:
INFO:__main__:Train Loss: 0.6937, Val Loss: 0.6978
INFO:__main__:Epoch 3/50:
INFO:__main__:Train Loss: 0.6907, Val Loss: 0.7007
INFO:__main__:Epoch 4/50:
INFO:__main__:Train Loss: 0.6906, Val Loss: 0.6891
INFO:__main__:Epoch 5/50:
INFO:__main__:Train Loss: 0.6938, Val Loss: 0.6949
INFO:__main__:Epoch 6/50:
INFO:__main__:Train Loss: 0.6891, Val Loss: 0.7045
INFO:__main__:Epoch 7/50:
INFO:__main__:Train Loss: 0.6895, Val Loss: 0.7023
INFO:__main__:Epoch 8/50:
INFO:__main__:Train Loss: 0.6867, Val Loss: 0.7015
INFO:__main__:Epoch 9/50:
INFO:__main__:Train Loss: 0.6868, Val Loss: 0.7102
INFO:__main__:Epoch 10/50:
INFO:__main__:Train Loss: 0.6890, Val Loss: 0.6994
INFO:__main__:Epoch 11/50:
INFO:__main__:Train Loss: 0.6845, Val Loss: 0.7115
INFO:__main__:Epoch 12/50:
INFO:__main__:Train Loss: 0.6805, Val Loss: 0.7062
INFO:__main__:Epoch 13/50:
INFO:__main__:Train Loss: 0.6836, Val Loss: 0.